In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.interpolate import interp1d
from scipy.spatial import ConvexHull
from scipy.signal import savgol_filter
import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go
import sys
import logging

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

sys.path.append("../components/paddock/")

%load_ext autoreload
%autoreload 2
from telemetry.influx import Influx
from telemetry.analyzer import Analyzer

from IPython.display import Image

pd.set_option("display.max_columns", None)

influx = Influx()
influx.laps_from_file("oulton international.csv")
analyzer = Analyzer()

In [12]:
# (game, session, track, car) = "iRacing,1670222575,pocono 2016,Dallara IR18".split(",")
# (game, session, track, car) = "Automobilista 2,1671635969,Cascavel4:Cascavel2,Mercedes-Benz Actros".split(",")
# (game, session, track, car) = "iRacing,1670287161,spielberg gp,Porsche 911 GT3.R".split(",")
# (game, session, track, car) = "iRacing,1670238139,spa up,Mercedes W12".split(",")
# (game, session, track, car) = "iRacing,1671643370,charlotte 2018 2019 rallycrosslong,Volkswagen Beetle GRC Lite".split(",")
# (game, session, track, car) = "iRacing,1670846922,nurburgring nordschleife,Porsche 911 GT3 Cup (992)".split(",")
# (game, session, track, car) = "Automobilista 2,1670846922,Nurburgring_2020:Nordschleife_2020_24hr,Porsche Cayman GT4 Clubsport MR".split(",")
# (game, session, track, car) = "Assetto Corsa (64 bit),1670215352,simtraxx_zeran:rally1,porsche_550_1500_rs_spyder_s1".split(",")
# (game, session, track, car) = "iRacing,1670351687,fuji gp,Ferrari 488 GT3 Evo 2020".split(",")
(
    game,
    session,
    track,
    car,
) = "iRacing,1673613841,sebring international,Ferrari 488 GT3 Evo 2020".split(",")
(
    game,
    session,
    track,
    car,
) = "iRacing,1673537226,oulton international,Mazda MX-5 Cup".split(",")
all_laps = influx.telemetry_for(game=game, track=track, car=car)
all_laps = all_laps[0:1]

INFO:root:Processing iRacing oulton international : 1673537226 - 13 : 4289 - 115.029839
INFO:root:Processing iRacing oulton international : 1673550700 -  2 : 4289 - 115.0448
INFO:root:Processing iRacing oulton international : 1673550700 -  9 : 4289 - 115.057365


In [13]:
display(all_laps[0].head())
fig = go.Figure()

for df in all_laps:
    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Brake"],
        marker=dict(size=2),
        name=df["id"].iloc[0],
    )
fig.show()

,result,table,_start,_stop,_time,CarModel,CurrentLap,GameName,SessionId,SessionTypeName,TrackCode,_measurement,host,topic,user,Brake,Clutch,CurrentLapTime,DistanceRoundTrack,Gear,Handbrake,Rpms,SpeedMs,SteeringAngle,Throttle,id
0,_result,0,2023-01-12 16:11:26.609394+00:00,2023-01-12 16:13:21.619012+00:00,2023-01-12 16:11:26.648978+00:00,Mazda MX-5 Cup,13,iRacing,1673537226,Practice,oulton international,laps_cc,telegraf,crewchief/Jim/1673537226/iRacing/oulton intern...,Jim,0.0,1.0,115.435471,1.945577,3.0,0.0,0.0,41.750180,0.017062,1.0,1673537226-13
1,_result,0,2023-01-12 16:11:26.609394+00:00,2023-01-12 16:13:21.619012+00:00,2023-01-12 16:11:26.670645+00:00,Mazda MX-5 Cup,13,iRacing,1673537226,Practice,oulton international,laps_cc,telegraf,crewchief/Jim/1673537226/iRacing/oulton intern...,Jim,0.0,1.0,115.468800,3.336780,3.0,0.0,0.0,41.782402,0.025861,1.0,1673537226-13
2,_result,0,2023-01-12 16:11:26.609394+00:00,2023-01-12 16:13:21.619012+00:00,2023-01-12 16:11:26.702131+00:00,Mazda MX-5 Cup,13,iRacing,1673537226,Practice,oulton international,laps_cc,telegraf,crewchief/Jim/1673537226/iRacing/oulton intern...,Jim,0.0,1.0,115.502144,4.729015,3.0,0.0,0.0,41.817380,0.044791,1.0,1673537226-13
3,_result,0,2023-01-12 16:11:26.609394+00:00,2023-01-12 16:13:21.619012+00:00,2023-01-12 16:11:26.729081+00:00,Mazda MX-5 Cup,13,iRacing,1673537226,Practice,oulton international,laps_cc,telegraf,crewchief/Jim/1673537226/iRacing/oulton intern...,Jim,0.0,1.0,115.518806,5.425600,3.0,0.0,0.0,41.842470,0.055190,1.0,1673537226-13
4,_result,0,2023-01-12 16:11:26.609394+00:00,2023-01-12 16:13:21.619012+00:00,2023-01-12 16:11:26.760761+00:00,Mazda MX-5 Cup,13,iRacing,1673537226,Practice,oulton international,laps_cc,telegraf,crewchief/Jim/1673537226/iRacing/oulton intern...,Jim,0.0,1.0,115.552139,6.819941,3.0,0.0,0.0,41.890324,0.072255,1.0,1673537226-13


In [14]:
# remove laps that are too different from the others
laps = analyzer.remove_uncorrelated_laps(all_laps, column="Brake", threshold=0.5)

In [15]:
# get the max distance for each lap and select the max
max_distance = np.max([df["DistanceRoundTrack"].max() for df in laps])

display(f"max distance: {max_distance}")

'max distance: 4289.89063'

In [16]:
df = laps[0].copy()
df = analyzer.resample(df, freq=2)
display(df)

fig = go.Figure()
fig.add_scatter(
    x=df["DistanceRoundTrack"],
    y=df["Brake"],
    mode="lines",
    marker=dict(size=2),
    name="combined",
)

,DistanceRoundTrack,Brake,SpeedMs,id
0,1.945577,0.0,41.750180,1673537226-13
1,3.946485,0.0,41.797720,1673537226-13
2,5.947392,0.0,41.860378,1673537226-13
3,7.948300,0.0,41.927585,1673537226-13
4,9.949208,0.0,41.988914,1673537226-13
...,...,...,...,...
2139,4281.886999,0.0,42.321394,1673537226-13
2140,4283.887907,0.0,42.385082,1673537226-13
2141,4285.888815,0.0,42.448150,1673537226-13
2142,4287.889722,0.0,42.503412,1673537226-13


In [33]:
# create one plotly figure
fig = go.Figure()
all_minima = []
for df in laps:
    df = analyzer.drop_decreasing(df)
    df = analyzer.resample(df, freq=1)
    df = analyzer.extend_lap(df)

    # smooth the laps
    window_length = 20  # meters
    df["Brake"] = savgol_filter(df["Brake"], window_length, 2)

    # remove points less than 0
    # df = df[df["Brake"] >= 0]

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Brake"],
        mode="lines",
        marker=dict(size=2),
        name=df["id"].iloc[0],
    )
    extrema = analyzer.local_maxima(df, column="Brake", points=50)
    print(f"number of minima {len(extrema)} for lap {df['id'].iloc[0]}")
    all_minima.append(extrema)

    fig.add_scatter(
        x=extrema["DistanceRoundTrack"],
        y=extrema["Brake"],
        mode="markers",
        marker=dict(size=10),
        name=df["id"].iloc[0],
    )
    # break

fig.show()

number of minima 21 for lap 1673537226-13


In [18]:
fig = go.Figure()
df = laps[0]
df = analyzer.extend_lap(df)

fig.add_scatter(x=df["DistanceRoundTrack"], y=df["Brake"], marker=dict(size=2), name="ref")

centroids_df, labels = analyzer.cluster(all_minima, field="Brake")

# Plot the laps points with different colors for each cluster the colors are in lables
fig.add_scatter(
    y=df["Brake"],
    x=df["DistanceRoundTrack"],
    mode="markers",
    marker=dict(size=1, color=labels),
)
fig.add_scatter(
    y=df["SpeedMs"] / 50,
    x=df["DistanceRoundTrack"],
    mode="markers",
    marker=dict(size=1, color=labels),
)

# Add the centroids to the plot
fig.add_scatter(
    x=centroids_df["DistanceRoundTrack"],
    y=centroids_df["Brake"],
    mode="markers+text",
    marker=dict(size=10, color="red"),
)

In [19]:
# display(centroids_df)
# sort the centroids by DistanceRoundTrack and reset the index
centroids_df = centroids_df.sort_values(by=["DistanceRoundTrack"])
centroids_df = centroids_df.reset_index(drop=True)
df = centroids_df.copy()

# apply modulo length to DistanceRoundTrack
df["DistanceRoundTrack"] = df["DistanceRoundTrack"].mod(max_distance)

# sort the centroids by DistanceRoundTrack and reset the index
df = df.sort_values(by=["DistanceRoundTrack"])
df = df.reset_index(drop=True)
display(df)

n_clusters = int(len(df) / 3)
turns, labels = analyzer.cluster([df], field="Brake", n_clusters=n_clusters)
turns = turns.sort_values(by=["DistanceRoundTrack"])
turns = turns.reset_index(drop=True)
display(turns)

fig = go.Figure()
df = laps[0].copy()

fig.add_scatter(x=df["DistanceRoundTrack"], y=df["Brake"], marker=dict(size=2), name="Brake")

# Add the turns to the plot
fig.add_scatter(
    x=turns["DistanceRoundTrack"],
    y=turns["Brake"],
    mode="markers+text",
    marker=dict(size=10, color="red"),
)

fig.add_scatter(
    y=df["SpeedMs"] / 50,
    x=df["DistanceRoundTrack"],
    mode="markers",
    marker=dict(size=1, color=labels),
)

,Brake,DistanceRoundTrack
0,0.603660,206.942950
1,0.603660,206.942950
2,0.603660,206.942950
3,-0.000882,262.942233
4,-0.000882,262.942233
5,-0.000882,262.942233
6,0.460762,698.936646
7,0.460762,698.936646
8,0.460762,698.936646
9,1.005724,1552.925702


,Brake,DistanceRoundTrack
0,0.603660,206.942950
1,-0.000882,262.942233
2,0.460762,698.936646
3,1.005724,1552.925702
4,1.006733,2041.919436
5,1.019340,2571.912645
6,0.523301,3335.902855
7,1.007283,3908.895512


In [20]:
# for each turn
for i in range(len(turns)):
    # get the distance of the turn
    brake_max = turns["Brake"].iloc[i]
    distance = turns["DistanceRoundTrack"].iloc[i]
    # display(f'brake force {brake_max} at distance {distance}')

    brake_starts = []
    brake_starts_minus_5 = []
    for df in laps:
        # go back from the turn to find where Brake starts
        brake = brake_max
        # find the index in df where DistanceRoundTrack is equal to distance
        if len(df[df["DistanceRoundTrack"] > distance]) > 0:
            index = df[df["DistanceRoundTrack"] > distance].index[0]
            brake_start = distance
            while brake > 0.1:
                index -= 1
                if index < 0:
                    index = len(df) - 1
                brake = df["Brake"].iloc[index]
            brake_start = df["DistanceRoundTrack"].iloc[index]
            brake_starts.append(brake_start)

            # now go back 5.3 seconds
            # because we'll read "Brake in 3 2 1", which take 5.3 seconds
            current_lap_time = df["CurrentLapTime"].iloc[index]
            target_lap_time = current_lap_time - 5.3
            if target_lap_time < 0:
                index = len(df) - 1
                target_lap_time = df["CurrentLapTime"].max() - target_lap_time
                current_lap_time = df["CurrentLapTime"].iloc[index]

            cycles = 0
            while current_lap_time > target_lap_time and cycles <= 2:
                index -= 1
                if index < 0:
                    index = len(df) - 1
                    cycles += 1
                current_lap_time = df["CurrentLapTime"].iloc[index]

            if cycles != 2:
                brake_start_minus_5 = df["DistanceRoundTrack"].iloc[index]
                brake_starts_minus_5.append(brake_start_minus_5)

    brake_start = int(np.median(brake_starts))
    brake_start_minus_5 = int(np.median(brake_starts_minus_5))
    print(
        f"turn {i} brake peak at distance {distance} with force {brake_max} starting at to brake at {brake_start} announcing at {brake_start_minus_5}"
    )

turn 0 brake peak at distance 206.942950236152 with force 0.6036603311154383 starting at to brake at 176 announcing at 4289
turn 1 brake peak at distance 262.942232642906 with force -0.0008818768397425014 starting at to brake at 268 announcing at 4289
turn 2 brake peak at distance 698.936645666917 with force 0.4607617720233833 starting at to brake at 673 announcing at 442
turn 3 brake peak at distance 1552.9257023699117 with force 1.0057237488089228 starting at to brake at 1540 announcing at 1288
turn 4 brake peak at distance 2041.9194362431717 with force 1.0067327829977488 starting at to brake at 2008 announcing at 1802
turn 5 brake peak at distance 2571.9126447356616 with force 1.0193402181157438 starting at to brake at 2590 announcing at 2382
turn 6 brake peak at distance 3335.9028547135167 with force 0.5233012763647962 starting at to brake at 3368 announcing at 3136
turn 7 brake peak at distance 3908.8955121969075 with force 1.0072829655434246 starting at to brake at 3938 announcin